In [ ]:
!py -m pip install openai
!py -m pip install pinecone-client
!py -m pip install "pinecone-client[grpc]"
!py -m pip install langchain
!py -m pip install numpy
!py -m pip install langchain

In [42]:
import pandas as pd
df = pd.read_csv("law.csv")
df['Content'] = df['Content'].str.replace("\n", " ")
df

,Content
0,Chapter 1: Terminology Section 1. Terminology ...
1,Chapter 2: Application of Criminal Law Section...
2,Chapter 3: Punishments and Measures of Safety ...
3,Chapter 3: Punishments and Measures of Safety ...
4,Chapter 3: Punishments and Measures of Safety ...
...,...
70,Section 367. Refusal to Provide Information to...
71,Section 375. Obstruction to a Public Gutter or...
72,Section 381. Animal Cruelty Any person who mal...
73,Section 385. Obstruction of Public Way by Leav...


In [44]:
import pandas as pd
import re

# Extract sections using regular expressions
pattern = r'Section \d+\..*?(?=Section \d+\.|$)'
df['Section'] = df['Content'].apply(lambda x: re.findall(pattern, x, re.DOTALL))
# Explode the list of sections into separate rows
df = df.explode('Section')
df_text = df.drop('Content', axis= 1)
# # Reset index
df_text.reset_index(drop=True, inplace=True)
df = df_text.dropna()
# for row in df_text.Content:
#     if row < 20:
#         df_text.drop(row)

In [45]:
df

,Section
0,Section 1. Terminology and Definitions In this...
1,Section 2. Requirements for Application of Cri...
2,Section 3. Changes in Law If the law in force ...
3,Section 4. Jurisdiction Law shall apply to act...
4,Section 5. Partially Performed Actions The act...
...,...
411,Section 394. Bringing an animal into a Private...
412,Section 395. Caretaker Negligence Allows an An...
413,Section 396. Leaving Carcass in Public Way Any...
414,Section 397. Bullying in Public Any person who...


In [ ]:
for i in range(len(df.Section)):
    if len(df.Section.iloc[i]) < 20:
        df.drop(i)


In [ ]:
import pandas as pd
import os
# path = os.path.abspath("/content/drive/MyDrive/work/criminal-datasets.csv/")
df = pd.read_csv("criminal-datasets.csv")
df['text'] = df['text'].astype(str) + df['notes'].astype(str)
df['text'] = df['text'].str.replace('nan', '')
df = df.drop(['article','notes'], axis = 1)
df = df.drop(df.index[0:9])
df = df.reset_index()
df = df.drop('index', axis = 1)
df

In [113]:
openai.api_type = "azure"
openai.api_base = "https://fusion-gpt-training.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = "a83037e20d3d4d7982b0c52080e730fb"

In [46]:
import pinecone
from tqdm.auto import tqdm
import openai
import numpy as np
openai.api_type = "azure"
openai.api_base = "https://fusion-chatbot-demo.openai.azure.com/"
openai.api_version = "2023-05-15"
openai.api_key = "f07d23e10600494f8235496bbf5054b9"
pinecone.init(api_key="d9b8d145-bab6-406a-ac8a-e3a17d90b5b7", environment="us-west4-gcp-free")
index = pinecone.Index(index_name = "embed")
embed_model = "text-embedding-ada-002"

In [47]:
x = np.squeeze(df.Section.values)
sentence_list = [x for x in x]
sentence_list

['Section 1. Terminology and Definitions In this Code: “Dishonestly” means in such a manner as to enrich oneself or another by an unlawful means; “Public way” means a land course or watercourse for public transport, and includes a railway or tramway for public transit; “Public place” means any place to which the public is entitled to gain access; “Dwelling” means a place of habitation, such as an inhabited house, building, ship or floating house, and includes the vicinity of such place, whether encircled; “Weapon” includes an object which is not a weapon by nature but is used or intended to be used in causing any grievous bodily harm as if weapon; “Forcible attack” means an assault against the person or mind of a person, whether by physical force or by any other means, and includes any action by which a person places another in the state of being incapable of resistance, whether by intoxicant, hypnotization or any other similar means; “Document” means a paper or other object on which t

In [160]:
Vectorstore, Only_Vectorstore = [], [] # consist of 20 vectors of chunk (sentence1 , sentence2, sentence3, ... ,sentence20)
for idx in range(len(sentence_list)):
  embedding_ = openai.Embedding.create(input=sentence_list[idx],engine=embed_model)
  Vectorstore.append(embedding_)

for item in Vectorstore:
  for data_item in item['data']:
    Vector = data_item['embedding']
    Only_Vectorstore.append(Vector)

In [48]:
len(Only_Vectorstore)

NameError: name 'Only_Vectorstore' is not defined

In [49]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]
embed_list = []
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentence_list)
embed_list.append(embeddings.tolist())
print(embeddings)


[[ 0.00393651  0.05660619 -0.05797224 ...  0.03355067  0.04730756
  -0.03641209]
 [ 0.00125762  0.03853644  0.01663387 ...  0.01952292  0.0479911
  -0.1307762 ]
 [-0.05873283  0.07715406 -0.00967065 ... -0.00412422  0.0545597
  -0.07871969]
 ...
 [ 0.06280458  0.10383014  0.04420355 ...  0.05025899  0.02703384
  -0.01525837]
 [ 0.01138099  0.05973152  0.00203587 ... -0.05068376  0.03509418
  -0.06826778]
 [-0.01931517  0.11983897  0.02341572 ...  0.0049629   0.04153806
  -0.05672906]]


In [50]:
len(embed_list[0])

413

In [215]:
import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-roberta-large")
model = AutoModel.from_pretrained("princeton-nlp/sup-simcse-roberta-large")
torch.backends.cudnn.enabled = True
torch.cuda.is_available = lambda: True
torch.cuda.is_available()

batch_size = 1  # Set your desired batch size
num_sentences = 50
num_batches = (num_sentences + batch_size - 1) // batch_size
embed_list = []
for batch_idx in range(num_batches):
    start_idx = batch_idx * batch_size
    end_idx = min((batch_idx + 1) * batch_size, num_sentences)
    batch_sentences = sentence_list[start_idx:end_idx]

    inputs = tokenizer(
        batch_sentences, padding=True, truncation=True, return_tensors="pt", add_special_tokens=True
    )
    with torch.no_grad():
        embeddings = model(**inputs, output_hidden_states=True, return_dict=True).last_hidden_state
        embed_list.append(embeddings.tolist())
    # Calculate cosine similarities in batch
    cosine_sims = []
        for i in range(len(batch_sentences)):
            for j in range(i + 1, len(batch_sentences)):
                cosine_sim = 1 - cosine(embeddings[i], embeddings[j])
                cosine_sims.append((batch_sentences[i], batch_sentences[j], cosine_sim))

        # Print cosine similarities for this batch
        for sentence1, sentence2, cosine_sim in cosine_sims:
            print(
                "Cosine similarity between \"%s\" and \"%s\" is: %.3f"
                % (sentence1, sentence2, cosine_sim)
            )


In [217]:
import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer

# Import our models. The package will take care of downloading the models automatically
tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-roberta-large")
model = AutoModel.from_pretrained("princeton-nlp/sup-simcse-roberta-large")

# Tokenize input texts
texts = [
    df.Section.iloc[0],  
    df.Section.iloc[10],
    df.Section.iloc[0]
]
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# Get the embeddings
with torch.no_grad():
    embeddings = model(**inputs, output_hidden_states=True, return_dict=True).pooler_output

# Calculate cosine similarities
# Cosine similarities are in [-1, 1]. Higher means more similar
cosine_sim_0_1 = 1 - cosine(embeddings[0], embeddings[1])
cosine_sim_0_2 = 1 - cosine(embeddings[0], embeddings[2])

print("Cosine similarity between \"%s\" and \"%s\" is: %.3f" % (texts[0], texts[1], cosine_sim_0_1))
print("Cosine similarity between \"%s\" and \"%s\" is: %.3f" % (texts[0], texts[2], cosine_sim_0_2))

Cosine similarity between "Section 1. Terminology and Definitions In this Code: “Dishonestly” means in such a manner as to enrich oneself or another by an unlawful means; “Public way” means a land course or watercourse for public transport, and includes a railway or tramway for public transit; “Public place” means any place to which the public is entitled to gain access; “Dwelling” means a place of habitation, such as an inhabited house, building, ship or floating house, and includes the vicinity of such place, whether encircled; “Weapon” includes an object which is not a weapon by nature but is used or intended to be used in causing any grievous bodily harm as if weapon; “Forcible attack” means an assault against the person or mind of a person, whether by physical force or by any other means, and includes any action by which a person places another in the state of being incapable of resistance, whether by intoxicant, hypnotization or any other similar means; “Document” means a paper o

In [30]:
flat_list = [item for item in embed_list]
print(len(flat_list[0][0]))

384


In [64]:
import pandas as pd
Embedding = pd.DataFrame(pd.Series(embed_list[0]), columns = ['Embedding'])
Embedding['text'] = df.Section
Embedding = Embedding.dropna()
# df.insert(0, 'index', [str(k) for k in range(len(df))])

In [68]:
batch_size = 8 #int(len(Embedding['text'])/2)
for i in range(0, len(Embedding['text']), batch_size):
    # set end position of batch
    i_end = min(i+batch_size, len(Embedding['text']))
    # get batch of lines and IDs
    lines_batch = Embedding['text'][i: i+batch_size]
    ids_batch = [str(n) for n in range(i, i_end)]
    # prep metadata and upsert batch
    meta = [{'text': line} for line in lines_batch]
    to_upsert = list(zip(ids_batch, Embedding.Embedding, meta))
    # upsert to Pinecone
    index.upsert(vectors=list(to_upsert), namespace= 'law_eng_full')

In [65]:
index.delete(delete_all=True, namespace="law_eng_full")

{}

In [69]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'law_eng_full': {'vector_count': 410}},
 'total_vector_count': 410}

In [124]:
df.Section.iloc[-5]

'Section 394. Bringing an animal into a Private Garden, Field, or Farm Any person who chases, drives or brings any animal into a garden, farm or field of another which has been prepared for planting, which has been planted or which yields seedlings or fruits shall be liable to imprisonment for not more than one month, or a fine not exceeding one thousand baht or both. '

In [72]:
query_test, embed_list = df.Section.iloc[1], []
# inputs = tokenizer
#     query_test, padding=True, truncation=True, return_tensors="pt", add_special_tokens=True
# )
# with torch.no_grad():
#     embeddings = model(**inputs, output_hidden_states=True, return_dict=True).last_hidden_state
#     embed_list.append(embeddings.tolist())
# embedding_ = openai.Embedding.create(input=query_test,engine=embed_model)
# embed_list.append(embedding_)
# index.query(vector=embed_list[0]['data'][0]['embedding'], namespace = "law_eng test", top_k= 5, include_metadata = True)
embed_list = []
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(query_test)
embed_list.append(embeddings.tolist())
index.query(vector=embed_list[0], namespace = "law_eng_full", top_k= 5, include_metadata = True)

{'matches': [{'id': '193',
              'metadata': {'text': 'Section 189. Aiding and Abetting Whoever '
                                   'assists another person who commits or is '
                                   'alleged of having committed an offence '
                                   'which is not a petty offence so that such '
                                   'person may not be punished by giving him '
                                   'lodging, by hiding, or by assisting him by '
                                   'any means so that he may not be arrested, '
                                   'shall be punished with imprisonment not '
                                   'exceeding two years or fined not exceeding '
                                   'four thousand Baht, or both. '},
              'score': 1.00000012,
              'values': []},
             {'id': '393',
              'metadata': {'text': 'Section 379. Using a Weapon during a Fight '
                  

In [131]:
limit = 4000
def retrieve(user_input):
    res = openai.Embedding.create(
        input=[user_input],
        engine=embed_model
    )
    # retrieve from Pinecone
    xq = res['data'][0]['embedding']
    # get relevant contexts
    res = index.query(xq, top_k = 5, namespace = "law_thai", include_metadata=True)
    contexts = [
        x['metadata']['text'] for x in res['matches']
    ]
    # build our prompt with the retrieved contexts included
    prompt_start = (
        "LawBot, a GPT-based AI assistant designed specifically for answering legal questions and explaining complex legal terms.\
        Analysis the Question sector and provide a comprehensive and briefly answer in Answer section.The answer must address all aspects of the Question and support your argument with\
        appropriate legal references in Context section. Only answered by thai language only \n\n"+"Context:\n"
    )
    prompt_end = (
        f"\n\nQuestion: {user_input}\nAnswer:"
    )
    # append contexts until hitting limit
    for i in range(1, len(contexts)):
        if len("\n\n---\n\n".join(contexts[:i])) >= limit:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts[:i-1]) +
                prompt_end
            )
            break
        elif i == len(contexts)-1:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts) +
                prompt_end
            )
    return prompt

In [ ]:
มอเตอร์ไซด์สองคันชนกัน คันหนึ่งไม่ใส่หมวกกันน็อก อีกคันหนึ่งใส่หมวกกันน็อกแต่ขับย้อนศร ใครเป็นคนผิด

In [118]:
import requests, uuid, json

key = "041e6b79b96146eabafaf524d78911be"
endpoint = "https://api.cognitive.microsofttranslator.com"

location = "southeastasia"
path = '/translate'
constructed_url = endpoint + path

params = {
    'api-version': '3.0',
    'from': 'th',
    'to': ['en']
}

headers = {
    'Ocp-Apim-Subscription-Key': key,
    'Ocp-Apim-Subscription-Region': location,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}

def translator(input):
    body = [{
        'text': input
    }]
    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()
    return response[0]["translations"][0]['text']


In [107]:
translator("เผาบ้านเรือนมีความผิดตามข้อไหน")

"What's the offense of burning houses?"

In [132]:
setting = [{"role":"system","content":""},{"role":"user","content":""}]
def gpt3_completion(user_input, query_with_contexts, engine='gpt-35-turbo-16k', temp=0.5, top_p=1.0, tokens=2000, freq_pen=0.0, pres_pen=0.2):
    setting.append({'role':'system',"content":query_with_contexts})
    setting.append({'role':'user',"content":f"{gpt3_sum(user_input)}"})
    response = openai.ChatCompletion.create(engine=engine, temperature=temp, max_tokens=tokens,
      top_p=top_p, frequency_penalty=freq_pen, presence_penalty=pres_pen, messages = setting)
    print("LawBot:", response['choices'][0]['message']['content'])
    
def gpt3_sum(user_input, engine='gpt-35-turbo', temp=0.5, top_p=0.95, tokens=2000, freq_pen=0.0, pres_pen=0.2):
    setting.append({'role':'system',"content":"Summary the user input as a bullet , use only thai language"})
    setting.append({'role':'user',"content":f"{user_input}"})
    response = openai.ChatCompletion.create(engine=engine, temperature=temp, max_tokens=tokens,
      top_p=top_p, frequency_penalty=freq_pen, presence_penalty=pres_pen, messages = setting)
    # print("SumBot:", response['choices'][0]['message']['content'])
    return response['choices'][0]['message']['content']

while True:
  # เผาบ้านเรือนมีความผิดตามข้อไหน
  # ป้องกันตัวเองอย่างไรไม่ให้ผิดกฏหมาย
  user_input = input()
  print("User:", user_input)
  if user_input == "quit":
    break
  query = retrieve(translator(user_input))
  # gpt3_sum(translator(user_input))
  response = gpt3_completion(translator(user_input), query)

User:  เผาบ้านเรือนมีความผิดตามข้อไหน
LawBot: การเผาบ้านถือเป็นความผิดตามมาตรา ๑๘๔ ของพระราชบัญญัติแก้ไขเพิ่มเติมประมวลกฎหมายอาญา (ฉบับที่ ๒๖) พ.ศ. ๒๕๖๐ ซึ่งกำหนดให้ผู้กระทำความผิดต้องรับโทษจำคุกไม่เกินห้าปี หรือปรับไม่เกินหนึ่งแสนบาท หรือทั้งจำทั้งปรับ โดยขึ้นอยู่กับความเป็นระเบียบของศาลและความเหมาะสมของความผิดที่เกิดขึ้น การเผาบ้านถือเป็นการกระทำที่อันตรายและสร้างความเสียหายต่อผู้อื่น ซึ่งเป็นความผิดที่สำคัญและได้รับการกำหนดโทษอย่างเคร่งครัดในกฎหมายอาญาของประเทศไทย

เพื่อให้ได้ข้อมูลที่แม่นยำและเป็นทางการมากขึ้น เราสามารถอ้างอิงถึงข้อบังคับและพระราชบัญญัติที่เกี่ยวข้องเพิ่มเติมได้ โดยให้พิจารณาตามความเหมาะสมและเป็นไปตามความต้องการของกรณีที่เกิดขึ้น

- พระราชบัญญัติแก้ไขเพิ่มเติมประมวลกฎหมายอาญา (ฉบับที่ ๒๖) พ.ศ. ๒๕๖๐
- พระราชบัญญัติควบคุมอาวุธ วัตถุระเบิดและของเสีย พ.ศ. ๒๕๒๖
- พระราชบัญญัติควบคุมอาคาร พ.ศ. ๒๕๓๒

**โปรดทราบว่าข้อมูลที่ให้เป็นการสรุปและอ้างอิงเท่านั้น ควรปรึกษากฎหมายและทนายความเพื่อข้อมูลที่แม่นยำและเป็นทางการมากขึ้น**
User: d


InvalidRequestError: This model's maximum context length is 4096 tokens. However, your messages resulted in 5319 tokens. Please reduce the length of the messages.

In [2]:
import os
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = "https://fusion-chatbot-demo.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "f07d23e10600494f8235496bbf5054b9"

In [3]:
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.schema import HumanMessage

query = "ฆ่าคนตายมีความผิดไหม"
# completion llm
BASE_URL = "https://fusion-chatbot-demo.openai.azure.com/"
API_KEY = "f07d23e10600494f8235496bbf5054b9"
DEPLOYMENT_NAME = "gpt-35-turbo-16k"
model = AzureChatOpenAI(
    openai_api_base=BASE_URL,
    openai_api_version="2023-05-15",
    deployment_name=DEPLOYMENT_NAME,
    openai_api_key=API_KEY,
    openai_api_type="azure",
)
with get_openai_callback() as cb:
    print(model(
        [
            HumanMessage(
                content="นายชื่ออะไร"
            )
        ]
    )
          )


content='ขออภัย ฉันไม่มีชื่อเนื่องจากฉันเป็นแค่ระบบคอมพิวเตอร์' additional_kwargs={} example=False


In [8]:
from langchain.prompts import PromptTemplate
template = """The following is a friendly conversation between a human and an AI.
Current conversation:
{history}
Human: {input}
AI Assistant:"""
prompt = PromptTemplate(input_variables=["history", "input"], template=template)

In [37]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory

def memory(input):
    memory=ConversationSummaryBufferMemory(llm=model)
    memory.save_context({"input": "hi"}, {"output": "whats up"})

def talk(user_input):
    conversation_with_summary = ConversationChain(
        llm=model,
        memory=memory,
        prompt=prompt,
        verbose=True,
    )
    print(conversation_with_summary.predict(input=user_input))

In [10]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory

class Chatbot():
    def __init__(self):
        self.memory = ConversationSummaryBufferMemory(llm=model)
    def talk(self, user_input):
        conversation_with_summary = ConversationChain(
            llm=model,
            memory= self.memory,
            prompt=prompt,
            verbose=True,
        )
        # self.memory.save_context({"input": user_input}, 
        #                          {"output": conversation_with_summary.predict(input=user_input)})
        print(conversation_with_summary.predict(input=user_input))


In [11]:
x = Chatbot()


In [13]:
x.talk("What did i ask you?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI.
Current conversation:
Human: 1 + 1 = ?
AI: 1 + 1 = 2.
Human: What did i ask you?
AI Assistant:

> Finished chain.
You asked me to solve the equation 1 + 1.
